In [ ]:
from itertools import chain
from tqdm import tqdm
import pandas as pd
import json
import numpy as np

In [ ]:
df = pd.read_csv('./tmp/out/sampleCsv.csv', dtype='object')


In [ ]:
df.head()

In [ ]:
variant_cols = [col for col in df.columns if col.endswith('__VARIANT')]

In [ ]:
final_dfs = []
for i, col in enumerate(variant_cols):
    print(f'{i+1}/{len(variant_cols)}')
    # defining col name
    col_simple = col.removesuffix('__VARIANT')
    
    # null values in col
    df_col_null = df.loc[df[col].isna()]
    df_col_null = df_col_null.drop(columns=[col]).to_dict(orient='records')
    
    # values in col
    df_col_variant = df.loc[df[col].notna()]
    
    if len(df_col_variant) > 0:
    
        # exploding each record
        col_dfs = []
        for index, record in tqdm(df_col_variant.iterrows(), total=len(df_col_variant)):
            # columns without target (base cols)
            record = record.to_dict()
            base_records = {i: record[i] for i in record if i != col}

            # loading json
            json_str = record[col].replace('\'', '"')
            try:
                json_struct = json.loads(json_str)
            except:
                print(json_str)

            # renaming target cols
            json_struct = [{f'{col_simple}_{k}' if 'SEC_TYOE' not in str(k).upper() else str(k): v for k, v in d.items()}
                        for d in json_struct]

            # adding base cols
            for d in json_struct:
                d.update(base_records)

            # adding exploded records
            final_dfs.extend(json_struct)
    final_dfs.extend(df_col_null)    


In [ ]:
merged = pd.DataFrame(final_dfs, dtype='object')

    

In [ ]:
merged.head()

In [ ]:
merged = merged.drop(columns=variant_cols)


In [ ]:
merged.head()

In [ ]:
len(merged)

In [ ]:
len(df)

In [ ]:
len(df.columns)

In [ ]:
len(merged.columns)

In [ ]:
merged.to_csv('./tmp/out/SampleCsv_FLAT.csv', index=False)
